# Parsing Social Media API's into CSV's

### General: 

In [ ]:
# Requests allows us to make HTTP requests which we will use to get data from an API
import requests
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
# NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np
# Datetime is a library that allows us to represent dates
import datetime

# Setting this option will print all collumns of a dataframe
pd.set_option('display.max_columns', None)
# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

In [ ]:
"""    
    Takes the dataset and uses whatever column is input in the first set of empty '' to call the API
    and append the data to the list.
    The second set of quotes will be the url to the specific column in the api
    The third set of quotes is the data you want from the api to be in that column
    
    Please Rename Method
    A method will be needed for each column
"""
def getNameOfColumn(data):
    for x in data['1']:
           if x:
            response = requests.get("2"+str(x)).json()
            NameOfColumn.append(response['3'])

In [ ]:
# This will have the API url
url = ''

In [ ]:
response = requests.get(url)

In [ ]:
# this will check that the request returned something, Should return 200
response.status_code

In [ ]:
# this will have the url to the API json
static_json_url=''

In [ ]:
# Uses json_normalize meethod to convert the json result into a dataframe
data = pd.json_normalize(response.json())

In [ ]:
# Get the head of a dataframe
data.head()

In [ ]:
# this will convert a dataframe to a csv
df.to_csv('.csv', index=False)

### Twitter: